# 7. Merge Final con Puntos de Interés (POIs)

Este notebook toma los archivos generados por `06.Merge_additional_files_duckdb.ipynb` y los enriquece con datos de puntos de interés (escuelas, hospitales, metro, etc.) utilizando una estrategia de vecino más cercano por coordenadas.

El proceso es el siguiente:
1. Itera sobre una lista de fechas para procesar múltiples archivos base.
2. Para cada archivo base, carga una lista de CSVs con puntos de interés (POIs).
3. Construye un índice geoespacial (`BallTree`) para cada conjunto de POIs para búsquedas eficientes.
4. Procesa el archivo base en fragmentos (chunks) para controlar el uso de memoria.
5. Para cada registro del archivo base, busca el POI más cercano dentro de un radio de 5 km.
6. Agrega las columnas del POI encontrado y la distancia en metros al registro base.
7. Guarda el resultado final enriquecido en un nuevo archivo Parquet para cada fecha.

In [1]:
import os
from pathlib import Path
import duckdb
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
import gc

In [2]:
# ============== CONFIGURACIÓN GENERAL ==============

# Fechas de los archivos a procesar
DATES_TO_PROCESS = [
    "2025-08-31", 
    "2025-09-14", 
    "2025-09-20", 
    "2025-09-27", 
    "2025-10-05", 
    "2025-10-11"
]

# Rutas base
BASE_DIR = Path("../../data/processed/")
CSV_FOLDER = BASE_DIR / "csv"
OUTPUT_DIR = BASE_DIR / "final_datasets"
TMP_CHUNK_DIR = OUTPUT_DIR / "tmp_chunks"

# Archivos CSV de puntos de interés
CSVS_INTERES = [
    "escuelas_privadas/escuelas_privadas_con_coordenadas.csv",
    "hospitales_y_centros_de_salud/hospitales_y_centros_de_salud_con_coordenadas.csv",
    "mb_shp/Metrobus_estaciones_con_coordenadas.csv",
    "stcmetro_shp/STC_Metro_estaciones_utm14n_con_coordenadas.csv",
    "areas_verdes_filtrado.csv",
    "escuelas_publicas.csv"
]

# Parámetros del proceso
CHUNK_SIZE = 50_000
MAX_DISTANCE_KM = 5.0
R_EARTH_M = 6_371_000.0
MAX_DISTANCE_RAD = (MAX_DISTANCE_KM * 1000.0) / R_EARTH_M

# Crear directorios de salida
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
TMP_CHUNK_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# ============== PREPARAR ÍNDICES DE CSVs (BallTree) ==============
print("🛠️ Preparando índices para los CSVs de puntos de interés...")

csv_models = []
for rel_path in CSVS_INTERES:
    csv_path = (CSV_FOLDER / rel_path).resolve()
    alias = csv_path.stem.lower().replace(" ", "_")
    if not csv_path.exists():
        print(f"⚠️ CSV no encontrado, se omite: {csv_path}")
        continue

    df = pd.read_csv(csv_path)
    if not {"latitud", "longitud"}.issubset(df.columns):
        print(f"⚠️ {csv_path.name} no tiene 'latitud' y 'longitud'. Se omite.")
        continue

    df = df.loc[df["latitud"].notna() & df["longitud"].notna()].copy()
    if df.empty:
        print(f"⚠️ {csv_path.name} sin coordenadas válidas. Se omite.")
        continue

    rename_map = {col: f"{col}_{alias}" for col in df.columns}
    df = df.rename(columns=rename_map)

    coords_deg = df[[f"latitud_{alias}", f"longitud_{alias}"]].astype(float).values
    coords_rad = np.radians(coords_deg)
    tree = BallTree(coords_rad, metric="haversine")

    csv_models.append({
        "alias": alias,
        "df": df,
        "tree": tree
    })
    print(f"✅ Índice listo: {alias} ({len(df):,} puntos)")

if not csv_models:
    raise RuntimeError("❌ No hay CSVs válidos para procesar. Abortando.")

🛠️ Preparando índices para los CSVs de puntos de interés...
✅ Índice listo: escuelas_privadas_con_coordenadas (3,659 puntos)
✅ Índice listo: hospitales_y_centros_de_salud_con_coordenadas (27 puntos)
✅ Índice listo: metrobus_estaciones_con_coordenadas (324 puntos)
✅ Índice listo: stc_metro_estaciones_utm14n_con_coordenadas (195 puntos)
✅ Índice listo: areas_verdes_filtrado (2,750 puntos)
✅ Índice listo: escuelas_publicas (2,242 puntos)


In [4]:
# ============== BUCLE PRINCIPAL DE PROCESAMIENTO POR FECHA ==============

for run_date in DATES_TO_PROCESS:
    print(f"\n\n{'='*60}")
    print(f"🚀 Iniciando procesamiento para la fecha: {run_date}")
    print(f"{'='*60}")

    # --- 1. Definir rutas de entrada y salida ---
    base_parquet = BASE_DIR / f"merged_inmuebles24_departamentos_duckdb_cp_{run_date}.parquet"
    output_final = OUTPUT_DIR / f"final_merged_inmuebles24_{run_date}.parquet"

    if not base_parquet.exists():
        print(f"❌ Archivo base no encontrado para la fecha {run_date}, omitiendo: {base_parquet}")
        continue

    # --- 2. Conectar a DuckDB y obtener metadatos ---
    con = duckdb.connect()
    try:
        n_rows = con.execute(f"SELECT COUNT(*) FROM read_parquet('{base_parquet.as_posix()}')").fetchone()[0]
        print(f"📦 Archivo base: {base_parquet.name} ({n_rows:,} filas)")
        
        sample = con.execute(f"SELECT * FROM read_parquet('{base_parquet.as_posix()}') LIMIT 1").fetchdf()
        if not {"longitud", "latitud"}.issubset(sample.columns):
            raise ValueError("El archivo base debe tener 'longitud' y 'latitud'.")
    except Exception as e:
        print(f"❌ Error al leer el archivo base {base_parquet.name}: {e}")
        con.close()
        continue

    # --- 3. Procesar en chunks ---
    chunk_paths = []
    for start in range(0, n_rows, CHUNK_SIZE):
        end = min(start + CHUNK_SIZE, n_rows)
        print(f"\n🧩 Chunk {start:,} → {end:,} ({end-start:,} filas)")

        base_chunk = con.execute(
            f"SELECT * FROM read_parquet('{base_parquet.as_posix()}') LIMIT {end-start} OFFSET {start}"
        ).fetchdf()

        base_coords_rad = np.radians(base_chunk[["latitud", "longitud"]].astype(float).values)

        for model in csv_models:
            alias = model["alias"]
            tree = model["tree"]
            df_csv = model["df"]

            dist_rad, idx = tree.query(base_coords_rad, k=1)
            dist_rad, idx = dist_rad.flatten(), idx.flatten()

            within_radius = dist_rad <= MAX_DISTANCE_RAD
            matched_data = df_csv.iloc[idx].reset_index(drop=True)

            # Agregar distancia en metros
            dist_m = dist_rad * R_EARTH_M
            base_chunk[f"dist_m_{alias}"] = np.where(within_radius, dist_m, np.nan)

            # Agregar coordenadas del POI
            lat_col_poi = f"latitud_{alias}"
            lon_col_poi = f"longitud_{alias}"

            lat_vals = matched_data[lat_col_poi].copy()
            lat_vals[~within_radius] = np.nan
            base_chunk[f"poi_lat_{alias}"] = lat_vals.values

            lon_vals = matched_data[lon_col_poi].copy()
            lon_vals[~within_radius] = np.nan
            base_chunk[f"poi_lon_{alias}"] = lon_vals.values

            print(f"   ➕ {alias}: {within_radius.sum():,} matches")

        out_path = TMP_CHUNK_DIR / f"merged_chunk_{run_date}_{start:08d}.parquet"
        base_chunk.to_parquet(out_path, index=False)
        chunk_paths.append(out_path)
        del base_chunk
        gc.collect()

    con.close()

    # --- 4. Unir chunks y guardar final ---
    if not chunk_paths:
        print("🤷 No se generaron chunks, no hay nada que unir.")
        continue
        
    print("\n📦 Combinando chunks para el archivo final...")
    final_df = pd.concat((pd.read_parquet(p) for p in chunk_paths), ignore_index=True)
    final_df.to_parquet(output_final, index=False)
    print(f"🎯 Archivo final guardado: {output_final.name} ({len(final_df):,} filas)")

    # --- 5. Limpiar chunks temporales ---
    for p in chunk_paths:
        p.unlink()
    print("🧹 Chunks temporales eliminados.")

print("\n\n🎉 Proceso completado para todas las fechas.")



🚀 Iniciando procesamiento para la fecha: 2025-08-31
📦 Archivo base: merged_inmuebles24_departamentos_duckdb_cp_2025-08-31.parquet (4,094 filas)

🧩 Chunk 0 → 4,094 (4,094 filas)
   ➕ escuelas_privadas_con_coordenadas: 4,047 matches
   ➕ hospitales_y_centros_de_salud_con_coordenadas: 3,947 matches
   ➕ metrobus_estaciones_con_coordenadas: 3,574 matches
   ➕ stc_metro_estaciones_utm14n_con_coordenadas: 3,530 matches
   ➕ areas_verdes_filtrado: 4,047 matches
   ➕ escuelas_publicas: 4,047 matches

📦 Combinando chunks para el archivo final...
🎯 Archivo final guardado: final_merged_inmuebles24_2025-08-31.parquet (4,094 filas)
🧹 Chunks temporales eliminados.


🚀 Iniciando procesamiento para la fecha: 2025-09-14
📦 Archivo base: merged_inmuebles24_departamentos_duckdb_cp_2025-09-14.parquet (4,197 filas)

🧩 Chunk 0 → 4,197 (4,197 filas)
   ➕ escuelas_privadas_con_coordenadas: 4,145 matches
   ➕ hospitales_y_centros_de_salud_con_coordenadas: 4,025 matches
   ➕ metrobus_estaciones_con_coordenadas